# Your First AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) from Oxford of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load the image dataset and create a pipeline.
* Build and Train an image classifier on this dataset.
* Use your trained model to perform inference on flower images.

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

## Import Resources

In [1]:
# The new version of dataset is only available in the tfds-nightly package.
%pip --no-cache-dir install tensorflow-datasets --user
# DON'T MISS TO RESTART THE KERNEL


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import TensorFlow 
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [3]:
# TODO: Make all other necessary imports.

In [4]:
print('Using:')
print('\t\u2022 TensorFlow version:', tf.__version__)
print('\t\u2022 tf.keras version:', tf.keras.__version__)
print('\t\u2022 Running on GPU' if tf.test.is_gpu_available() else '\t\u2022 GPU device not found. Running on CPU')

Using:
	• TensorFlow version: 2.0.0
	• tf.keras version: 2.2.4-tf
	• Running on GPU


## Load the Dataset

Here you'll use `tensorflow_datasets` to load the [Oxford Flowers 102 dataset](https://www.tensorflow.org/datasets/catalog/oxford_flowers102). This dataset has 3 splits: `'train'`, `'test'`, and `'validation'`.  You'll also need to make sure the training data is normalized and resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet, but you'll still need to normalize and resize the images to the appropriate size.

In [6]:
# Download data to default local directory "~/tensorflow_datasets"
!python -m tensorflow_datasets.scripts.download_and_prepare --register_checksums=False --datasets=oxford_flowers102


I1007 07:10:18.257577 139947807663872 download_and_prepare.py:171] Running download_and_prepare for datasets:
oxford_flowers102
I1007 07:10:18.257881 139947807663872 download_and_prepare.py:172] Version: "None"
I1007 07:10:18.258373 139947807663872 dataset_builder.py:187] Load pre-computed datasetinfo (eg: splits) from bucket.
I1007 07:10:18.299828 139947807663872 dataset_info.py:410] Loading info from GCS for oxford_flowers102/0.0.1
I1007 07:10:18.317020 139947807663872 download_and_prepare.py:122] download_and_prepare for dataset oxford_flowers102/0.0.1...
I1007 07:10:18.366949 139947807663872 dataset_builder.py:273] Generating dataset oxford_flowers102 (/root/tensorflow_datasets/oxford_flowers102/0.0.1)
Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]I1007 07:10:18.370339 139947807663872 download_manager.py:241] Downloading https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz into /root/tensorfl

Dl Size...:   9%|██▌                         | 30/328 [00:02<00:23, 12.61 MiB/s]

Extraction completed...: 0 file [00:02, ? file/s]
Dl Size...:   9%|██▋                         | 31/328 [00:02<00:19, 14.89 MiB/s]

Dl Size...:  10%|██▋                         | 32/328 [00:02<00:19, 14.89 MiB/s]

Dl Size...:  10%|██▊                         | 33/328 [00:02<00:19, 14.89 MiB/s]

Extraction completed...: 0 file [00:02, ? file/s]
Dl Size...:  10%|██▉                         | 34/328 [00:02<00:18, 15.69 MiB/s]

Dl Size...:  11%|██▉                         | 35/328 [00:02<00:18, 15.69 MiB/s]

Dl Size...:  11%|███                         | 36/328 [00:02<00:18, 15.69 MiB/s]

Dl Size...:  11%|███▏                        | 37/328 [00:02<00:18, 15.69 MiB/s]

Extraction completed...: 0 file [00:02, ? file/s]
Dl Size...:  12%|███▏                        | 38/328 [00:02<00:15, 18.64 MiB/s]

Dl Size...:  12%|███▎                        | 39/328 [00:02<00:15, 18.64 MiB/s]

Dl Size...:  12%|███▍         

Extraction completed...: 0 file [00:04, ? file/s]
Dl Size...:  23%|██████▍                     | 76/328 [00:04<00:08, 28.06 MiB/s]

Dl Size...:  23%|██████▌                     | 77/328 [00:04<00:08, 28.06 MiB/s]

Dl Size...:  24%|██████▋                     | 78/328 [00:04<00:08, 28.06 MiB/s]

Extraction completed...: 0 file [00:04, ? file/s]
Dl Size...:  24%|██████▋                     | 79/328 [00:04<00:08, 28.40 MiB/s]

Dl Size...:  24%|██████▊                     | 80/328 [00:04<00:08, 28.40 MiB/s]

Dl Size...:  25%|██████▉                     | 81/328 [00:04<00:08, 28.40 MiB/s]

Dl Size...:  25%|███████                     | 82/328 [00:04<00:08, 28.40 MiB/s]

Extraction completed...: 0 file [00:04, ? file/s]
Dl Size...:  25%|███████                     | 83/328 [00:04<00:08, 29.59 MiB/s]

Dl Size...:  26%|███████▏                    | 84/328 [00:04<00:08, 29.59 MiB/s]

Dl Size...:  26%|███████▎                    | 85/328 [00:04<00:08, 29.59 MiB/s]

Extraction completed...: 0 fil

Extraction completed...: 0 file [00:05, ? file/s]
Dl Size...:  37%|█████████▉                 | 121/328 [00:05<00:07, 26.64 MiB/s]

Dl Size...:  37%|██████████                 | 122/328 [00:05<00:07, 26.64 MiB/s]

Dl Size...:  38%|██████████▏                | 123/328 [00:05<00:07, 26.64 MiB/s]

Extraction completed...: 0 file [00:05, ? file/s]
Dl Size...:  38%|██████████▏                | 124/328 [00:05<00:07, 27.45 MiB/s]

Dl Size...:  38%|██████████▎                | 125/328 [00:05<00:07, 27.45 MiB/s]

Dl Size...:  38%|██████████▎                | 126/328 [00:06<00:07, 27.45 MiB/s]

Extraction completed...: 0 file [00:06, ? file/s]
Dl Size...:  39%|██████████▍                | 127/328 [00:06<00:07, 26.65 MiB/s]

Dl Size...:  39%|██████████▌                | 128/328 [00:06<00:07, 26.65 MiB/s]

Dl Size...:  39%|██████████▌                | 129/328 [00:06<00:07, 26.65 MiB/s]

Extraction completed...: 0 file [00:06, ? file/s]
Dl Size...:  40%|██████████▋                | 130/328 [00:06<0

Extraction completed...: 0 file [00:07, ? file/s]
Dl Size...:  51%|█████████████▋             | 166/328 [00:07<00:05, 28.37 MiB/s]

Dl Size...:  51%|█████████████▋             | 167/328 [00:07<00:05, 28.37 MiB/s]

Dl Size...:  51%|█████████████▊             | 168/328 [00:07<00:05, 28.37 MiB/s]

Extraction completed...: 0 file [00:07, ? file/s]
Dl Size...:  52%|█████████████▉             | 169/328 [00:07<00:05, 28.63 MiB/s]

Dl Size...:  52%|█████████████▉             | 170/328 [00:07<00:05, 28.63 MiB/s]

Dl Size...:  52%|██████████████             | 171/328 [00:07<00:05, 28.63 MiB/s]

Dl Size...:  52%|██████████████▏            | 172/328 [00:07<00:05, 28.63 MiB/s]

Extraction completed...: 0 file [00:07, ? file/s]
Dl Size...:  53%|██████████████▏            | 173/328 [00:07<00:05, 27.35 MiB/s]

Dl Size...:  53%|██████████████▎            | 174/328 [00:07<00:05, 27.35 MiB/s]

Dl Size...:  53%|██████████████▍            | 175/328 [00:07<00:05, 27.35 MiB/s]

Extraction completed...: 0 fil

Extraction completed...: 0 file [00:08, ? file/s]
Dl Size...:  64%|█████████████████▎         | 211/328 [00:09<00:04, 28.91 MiB/s]

Dl Size...:  65%|█████████████████▍         | 212/328 [00:09<00:04, 28.91 MiB/s]

Dl Size...:  65%|█████████████████▌         | 213/328 [00:09<00:03, 28.91 MiB/s]

Extraction completed...: 0 file [00:09, ? file/s]
Dl Size...:  65%|█████████████████▌         | 214/328 [00:09<00:03, 29.17 MiB/s]

Dl Size...:  66%|█████████████████▋         | 215/328 [00:09<00:03, 29.17 MiB/s]

Dl Size...:  66%|█████████████████▊         | 216/328 [00:09<00:03, 29.17 MiB/s]

Extraction completed...: 0 file [00:09, ? file/s]
Dl Size...:  66%|█████████████████▊         | 217/328 [00:09<00:03, 28.92 MiB/s]

Dl Size...:  66%|█████████████████▉         | 218/328 [00:09<00:03, 28.92 MiB/s]

Dl Size...:  67%|██████████████████         | 219/328 [00:09<00:03, 28.92 MiB/s]

Extraction completed...: 0 file [00:09, ? file/s]
Dl Size...:  67%|██████████████████         | 220/328 [00:09<0

Dl Size...:  78%|█████████████████████▏     | 257/328 [00:10<00:02, 29.89 MiB/s]

Dl Size...:  79%|█████████████████████▏     | 258/328 [00:10<00:02, 29.89 MiB/s]

Dl Size...:  79%|█████████████████████▎     | 259/328 [00:10<00:02, 29.89 MiB/s]

Extraction completed...: 0 file [00:10, ? file/s]
Dl Size...:  79%|█████████████████████▍     | 260/328 [00:10<00:02, 29.65 MiB/s]

Dl Size...:  80%|█████████████████████▍     | 261/328 [00:10<00:02, 29.65 MiB/s]

Dl Size...:  80%|█████████████████████▌     | 262/328 [00:10<00:02, 29.65 MiB/s]

Extraction completed...: 0 file [00:10, ? file/s]
Dl Size...:  80%|█████████████████████▋     | 263/328 [00:10<00:02, 29.49 MiB/s]

Dl Size...:  80%|█████████████████████▋     | 264/328 [00:10<00:02, 29.49 MiB/s]

Dl Size...:  81%|█████████████████████▊     | 265/328 [00:10<00:02, 29.49 MiB/s]

Extraction completed...: 0 file [00:10, ? file/s]
Dl Size...:  81%|█████████████████████▉     | 266/328 [00:10<00:02, 28.20 MiB/s]

Dl Size...:  81%|█████████████

Extraction completed...: 0 file [00:12, ? file/s]
Dl Size...:  92%|████████████████████████▊  | 302/328 [00:12<00:00, 28.50 MiB/s]

Dl Size...:  92%|████████████████████████▉  | 303/328 [00:12<00:00, 28.50 MiB/s]

Dl Size...:  93%|█████████████████████████  | 304/328 [00:12<00:00, 28.50 MiB/s]

Dl Size...:  93%|█████████████████████████  | 305/328 [00:12<00:00, 28.50 MiB/s]

Extraction completed...: 0 file [00:12, ? file/s]
Dl Size...:  93%|█████████████████████████▏ | 306/328 [00:12<00:00, 28.16 MiB/s]

Dl Size...:  94%|█████████████████████████▎ | 307/328 [00:12<00:00, 28.16 MiB/s]

Dl Size...:  94%|█████████████████████████▎ | 308/328 [00:12<00:00, 28.16 MiB/s]

Extraction completed...: 0 file [00:12, ? file/s]
Dl Size...:  94%|█████████████████████████▍ | 309/328 [00:12<00:00, 27.47 MiB/s]

Dl Size...:  95%|█████████████████████████▌ | 310/328 [00:12<00:00, 27.47 MiB/s]

Dl Size...:  95%|█████████████████████████▌ | 311/328 [00:12<00:00, 27.47 MiB/s]

Extraction completed...: 0 fil

In [6]:
# TODO: Load the dataset with TensorFlow Datasets. Hint: use tfds.load()
train_split = 60
test_val_split = 20

splits = tfds.Split.ALL.subsplit([train_split,test_val_split, test_val_split])

In [18]:
a,b = tfds.load('oxford_flowers102', data_dir="102flowers.tgz", download=False)


AssertionError: Dataset oxford_flowers102: could not find data in 102flowers.tgz. Please make sure to call dataset_builder.download_and_prepare(), or pass download=True to tfds.load() before trying to access the tf.data.Dataset object.

In [9]:
# TODO: Create a training set, a validation set and a test set.
#(training_set, validation_set, test_set), dataset_info = tfds.load('oxford_flowers102', split=splits, as_supervised=True, with_info=True, download=True)
dataset, info = tfds.load("oxford_flowers102",with_info=True,download=False)

AssertionError: Dataset oxford_flowers102: could not find data in /root/tensorflow_datasets. Please make sure to call dataset_builder.download_and_prepare(), or pass download=True to tfds.load() before trying to access the tf.data.Dataset object.

## Explore the Dataset

In [5]:
dataset_info

# TODO: Get the number of examples in each set from the dataset info.

# TODO: Get the number of classes in the dataset from the dataset info.

NameError: name 'dataset_info' is not defined

In [ ]:
# TODO: Print the shape and corresponding label of 3 images in the training set.


In [ ]:
# TODO: Plot 1 image from the training set. 

# Set the title of the plot to the corresponding image label. 

### Label Mapping

You'll also need to load in a mapping from label to category name. You can find this in the file `label_map.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/3.7/library/json.html). This will give you a dictionary mapping the integer coded labels to the actual names of the flowers.

In [ ]:
with open('label_map.json', 'r') as f:
    class_names = json.load(f)

In [ ]:
# TODO: Plot 1 image from the training set. Set the title 
# of the plot to the corresponding class name. 



## Create Pipeline

In [ ]:
# TODO: Create a pipeline for each set.



# Build and Train the Classifier

Now that the data is ready, it's time to build and train the classifier. You should use the MobileNet pre-trained model from TensorFlow Hub to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! 

Refer to the rubric for guidance on successfully completing this section. Things you'll need to do:

* Load the MobileNet pre-trained network from TensorFlow Hub.
* Define a new, untrained feed-forward network as a classifier.
* Train the classifier.
* Plot the loss and accuracy values achieved during training for the training and validation set.
* Save your trained model as a Keras model. 

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right.

**Note for Workspace users:** One important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module. Also, If your model is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# TODO: Build and train your network.



In [ ]:
# TODO: Plot the loss and accuracy values achieved during training for the training and validation set.



## Testing your Network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Print the loss and accuracy values achieved on the entire test set.



## Save the Model

Now that your network is trained, save the model so you can load it later for making inference. In the cell below save your model as a Keras model (*i.e.* save it as an HDF5 file).

In [ ]:
# TODO: Save your trained model as a Keras model.



## Load the Keras Model

Load the Keras model you saved above.

In [ ]:
# TODO: Load the Keras model



# Inference for Classification

Now you'll write a function that uses your trained network for inference. Write a function called `predict` that takes an image, a model, and then returns the top $K$ most likely class labels along with the probabilities. The function call should look like: 

```python
probs, classes = predict(image_path, model, top_k)
```

If `top_k=5` the output of the `predict` function should be something like this:

```python
probs, classes = predict(image_path, model, 5)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

Your `predict` function should use `PIL` to load the image from the given `image_path`. You can use the [Image.open](https://pillow.readthedocs.io/en/latest/reference/Image.html#PIL.Image.open) function to load the images. The `Image.open()` function returns an `Image` object. You can convert this `Image` object to a NumPy array by using the `np.asarray()` function.

The `predict` function will also need to handle pre-processing the input image such that it can be used by your model. We recommend you write a separate function called `process_image` that performs the pre-processing. You can then call the `process_image` function from the `predict` function. 

### Image Pre-processing

The `process_image` function should take in an image (in the form of a NumPy array) and return an image in the form of a NumPy array with shape `(224, 224, 3)`.

First, you should convert your image into a TensorFlow Tensor and then resize it to the appropriate size using `tf.image.resize`.

Second, the pixel values of the input images are typically encoded as integers in the range 0-255, but the model expects the pixel values to be floats in the range 0-1. Therefore, you'll also need to normalize the pixel values. 

Finally, convert your image back to a NumPy array using the `.numpy()` method.

In [ ]:
# TODO: Create the process_image function



To check your `process_image` function we have provided 4 images in the `./test_images/` folder:

* cautleya_spicata.jpg
* hard-leaved_pocket_orchid.jpg
* orange_dahlia.jpg
* wild_pansy.jpg

The code below loads one of the above images using `PIL` and plots the original image alongside the image produced by your `process_image` function. If your `process_image` function works, the plotted image should be the correct size. 

In [ ]:
from PIL import Image

image_path = './test_images/hard-leaved_pocket_orchid.jpg'
im = Image.open(image_path)
test_image = np.asarray(im)

processed_test_image = process_image(test_image)

fig, (ax1, ax2) = plt.subplots(figsize=(10,10), ncols=2)
ax1.imshow(test_image)
ax1.set_title('Original Image')
ax2.imshow(processed_test_image)
ax2.set_title('Processed Image')
plt.tight_layout()
plt.show()

Once you can get images in the correct format, it's time to write the `predict` function for making inference with your model.

### Inference

Remember, the `predict` function should take an image, a model, and then returns the top $K$ most likely class labels along with the probabilities. The function call should look like: 

```python
probs, classes = predict(image_path, model, top_k)
```

If `top_k=5` the output of the `predict` function should be something like this:

```python
probs, classes = predict(image_path, model, 5)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

Your `predict` function should use `PIL` to load the image from the given `image_path`. You can use the [Image.open](https://pillow.readthedocs.io/en/latest/reference/Image.html#PIL.Image.open) function to load the images. The `Image.open()` function returns an `Image` object. You can convert this `Image` object to a NumPy array by using the `np.asarray()` function.

**Note:** The image returned by the `process_image` function is a NumPy array with shape `(224, 224, 3)` but the model expects the input images to be of shape `(1, 224, 224, 3)`. This extra dimension represents the batch size. We suggest you use the `np.expand_dims()` function to add the extra dimension. 

In [ ]:
# TODO: Create the predict function



# Sanity Check

It's always good to check the predictions made by your model to make sure they are correct. To check your predictions we have provided 4 images in the `./test_images/` folder:

* cautleya_spicata.jpg
* hard-leaved_pocket_orchid.jpg
* orange_dahlia.jpg
* wild_pansy.jpg

In the cell below use `matplotlib` to plot the input image alongside the probabilities for the top 5 classes predicted by your model. Plot the probabilities as a bar graph. The plot should look like this:

<img src='assets/inference_example.png' width=600px>

You can convert from the class integer labels to actual flower names using `class_names`. 

In [ ]:
# TODO: Plot the input image along with the top 5 classes
